In [2]:
from collections import Counter, defaultdict
import math
import numpy as np
import os
#import pandas as pd
import re
from scipy.sparse import csr_matrix
#import urllib.request
#import zipfile


In [3]:
# Read in data from pandas
import pandas as pd

# This is used for fast string concatination
from io import StringIO

# Use nltk for valid words
import nltk
# Need to make hash 'dictionaries' from nltk for fast processing
import collections as co


import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)


In [4]:
# Read the input
d = pd.read_csv("F:\IITC\ADM\Extra\consumer_complaints.csv") # the consumer dataset is now a Pandas DataFrame
# Only interested in data with consumer complaints
d = d[d['Consumer complaint narrative'].notnull()]

In [5]:
print(d['Consumer complaint narrative'])

56777     Received Capital One charge card offer XXXX. A...
56834     I do n't know how they got my cell number. I t...
56884     I 'm a longtime member of Charter One Bank/RBS...
56894     After looking at my credit report, I saw a col...
56898     I received a call from a XXXX XXXX from XXXX @...
56910     Was not contacted 4 years later about some pri...
56911     Collection Consultants is reporting a collecti...
56914     I had my purse stolen in 2007. They never foun...
56917     I attempted to apply for a Discover Card Onlin...
56918     Continued attempts by XXXX XXXX XXXX to collec...
56920     This is a continuation of a previous issue wit...
56928     Going through a divorce, my ex and I were unab...
56930     I am assisted with my mortgage through an agen...
56939     i submitted a payment on XXXX XXXX 2015 for my...
56940     I recieved a notice from Midland Credit Manage...
56956     XXXX Card services was bought out by Capital O...
56965     I was reported late by Discove

In [6]:
print(d[1:5])

      Date received           Product                            Sub-product  \
56834     3/23/2015   Debt collection  Other (i.e. phone, health club, etc.)   
56884     3/23/2015       Credit card                                    NaN   
56894     3/23/2015  Credit reporting                                    NaN   
56898     3/23/2015   Debt collection                            Payday loan   

                                        Issue  \
56834     Improper contact or sharing of info   
56884                                 Rewards   
56894  Incorrect information on credit report   
56898     Improper contact or sharing of info   

                                   Sub-issue  \
56834      Contacted me after I asked not to   
56884                                    NaN   
56894                Information is not mine   
56898  Talked to a third party about my debt   

                            Consumer complaint narrative  \
56834  I do n't know how they got my cell number. I 

In [7]:
# We want a very fast way to concat strings.
#  Try += if you don't believe this method is faster.
s=StringIO()
d['Consumer complaint narrative'].apply(lambda x: s.write(x))

k=s.getvalue()
s.close()
k=k.lower()
k=k.split()


In [8]:
#nltk.download()
#k


In [9]:
print(len(k))

23090783


In [10]:
# Next only want valid strings
words = co.Counter(nltk.corpus.words.words())
print(words)

Counter({'shade': 2, 'silver': 2, 'agreement': 2, 'stiff': 2, 'comb': 2, 'poor': 2, 'view': 2, 'baby': 2, 'sex': 2, 'stem': 2, 'mother': 2, 'shut': 2, 'ear': 2, 'seem': 2, 'brush': 2, 'rough': 2, 'event': 2, 'current': 2, 'apple': 2, 'substance': 2, 'birth': 2, 'week': 2, 'yellow': 2, 'crack': 2, 'mist': 2, 'tight': 2, 'curtain': 2, 'end': 2, 'left': 2, 'soup': 2, 'to': 2, 'carriage': 2, 'married': 2, 'from': 2, 'attention': 2, 'point': 2, 'soft': 2, 'natural': 2, 'way': 2, 'transport': 2, 'why': 2, 'than': 2, 'medical': 2, 'rub': 2, 'mountain': 2, 'disgust': 2, 'goat': 2, 'example': 2, 'roof': 2, 'white': 2, 'have': 2, 'war': 2, 'long': 2, 'price': 2, 'about': 2, 'small': 2, 'that': 2, 'down': 2, 'sail': 2, 'well': 2, 'powder': 2, 'motion': 2, 'theory': 2, 'step': 2, 'hat': 2, 'suggestion': 2, 'house': 2, 'quick': 2, 'relation': 2, 'sign': 2, 'across': 2, 'ring': 2, 'orange': 2, 'hand': 2, 'reason': 2, 'plate': 2, 'wave': 2, 'wire': 2, 'parcel': 2, 'reaction': 2, 'dress': 2, 'guide': 

In [11]:
stopWords = co.Counter( nltk.corpus.stopwords.words() )

print(stopWords)

Counter({'de': 9, 'en': 8, 'me': 7, 'da': 6, 'i': 5, 'er': 5, 'se': 5, 'o': 5, 'du': 5, 'a': 5, 'ne': 5, 'te': 5, 'men': 4, 'sin': 4, 'her': 4, 'vi': 4, 'man': 4, 'om': 4, 'era': 4, 'over': 4, 'no': 4, 'han': 4, 'den': 4, 'tu': 4, 'for': 4, 'mi': 4, 'alle': 4, 'som': 4, 'dem': 4, 'der': 4, 'e': 4, 'in': 4, 'ha': 4, 'et': 4, 's': 3, 'på': 3, 'nos': 3, 'so': 3, 'as': 3, 'fui': 3, 'med': 3, 'ma': 3, 'mit': 3, 'un': 3, 'være': 3, 'har': 3, 'ja': 3, 'hun': 3, 'si': 3, 'min': 3, 'at': 3, 'um': 3, 'os': 3, 'eller': 3, 'was': 3, 'le': 3, 'hans': 3, 'kan': 3, 'det': 3, 'la': 3, 'nu': 3, 'que': 3, 'al': 3, 'es': 3, 'on': 3, 'skulle': 3, 'he': 3, 'din': 3, 'y': 3, 'var': 3, 'under': 3, 'ned': 2, 'jeg': 2, 'dette': 2, 'hem': 2, 'waren': 2, 'бойы': 2, 'sinä': 2, 'm': 2, 'bli': 2, 'das': 2, 've': 2, 'has': 2, 'vil': 2, 'die': 2, 'del': 2, 'seremos': 2, 'kun': 2, 'hoe': 2, 'majd': 2, 'ki': 2, 'қарай': 2, 'be': 2, 'meg': 2, 'оның': 2, 'doch': 2, 'og': 2, 'sois': 2, 't': 2, 'minä': 2, 'wie': 2, 'minkä'

In [12]:

k=[i for i in k if i in words and i not in stopWords]

s=" ".join(k)


c = co.Counter(k)

In [13]:
# Take a look at the 14 most common words
freq_words = c.most_common(100)

In [14]:
print(freq_words)

[('credit', 149874), ('account', 111729), ('would', 86423), ('loan', 71033), ('bank', 67045), ('payment', 66060), ('told', 65502), ('received', 55649), ('debt', 50996), ('mortgage', 49129), ('information', 47816), ('never', 46978), ('card', 43244), ('get', 43058), ('company', 42869), ('pay', 42626), ('report', 42590), ('sent', 41871), ('time', 40546), ('said', 38023), ('call', 37829), ('could', 37249), ('letter', 37183), ('back', 34942), ('made', 32776), ('phone', 31472), ('due', 31364), ('since', 31103), ('amount', 30154), ('money', 29278), ('still', 29043), ('even', 28263), ('collection', 28046), ('number', 27862), ('one', 26919), ('home', 26788), ('make', 25411), ('balance', 24535), ('us', 24510), ('late', 23372), ('days', 22921), ('new', 22364), ('interest', 22137), ('check', 21968), ('know', 19762), ('please', 19535), ('another', 19458), ('date', 19151), ('like', 18808), ('service', 18788), ('contact', 18720), ('help', 18549), ('stated', 18521), ('several', 17989), ('need', 17734)

In [15]:
voc = list
voc = [word[0] for word in freq_words] 

In [16]:
print(type(voc))
print(voc)

<class 'list'>
['credit', 'account', 'would', 'loan', 'bank', 'payment', 'told', 'received', 'debt', 'mortgage', 'information', 'never', 'card', 'get', 'company', 'pay', 'report', 'sent', 'time', 'said', 'call', 'could', 'letter', 'back', 'made', 'phone', 'due', 'since', 'amount', 'money', 'still', 'even', 'collection', 'number', 'one', 'home', 'make', 'balance', 'us', 'late', 'days', 'new', 'interest', 'check', 'know', 'please', 'another', 'date', 'like', 'service', 'contact', 'help', 'stated', 'several', 'need', 'name', 'month', 'send', 'times', 'first', 'last', 'request', 'without', 'customer', 'dispute', 'day', 'complaint', 'chase', 'take', 'went', 'provide', 'see', 'going', 'got', 'file', 'trying', 'modification', 'bill', 'financial', 'address', 'charge', 'able', 'original', 'receive', 'business', 'spoke', 'informed', 'work', 'every', 'insurance', 'provided', 'agency', 'statement', 'put', 'monthly', 'removed', 'fee', 'full', 'remove', 'process']


In [17]:
vocab = {}

for word in voc:
    #print(word)
    if word not in vocab.keys():
        vocab.setdefault(word,-1)


In [18]:
print(type(vocab))
print(vocab)


<class 'dict'>
{'first': -1, 'us': -1, 'service': -1, 'like': -1, 'company': -1, 'told': -1, 'send': -1, 'back': -1, 'amount': -1, 'remove': -1, 'payment': -1, 'information': -1, 'make': -1, 'please': -1, 'dispute': -1, 'days': -1, 'due': -1, 'account': -1, 'number': -1, 'name': -1, 'work': -1, 'call': -1, 'financial': -1, 'sent': -1, 'stated': -1, 'day': -1, 'able': -1, 'mortgage': -1, 'original': -1, 'bill': -1, 'phone': -1, 'customer': -1, 'need': -1, 'another': -1, 'fee': -1, 'got': -1, 'late': -1, 'statement': -1, 'file': -1, 'removed': -1, 'loan': -1, 'never': -1, 'process': -1, 'letter': -1, 'know': -1, 'see': -1, 'business': -1, 'credit': -1, 'times': -1, 'still': -1, 'help': -1, 'modification': -1, 'get': -1, 'interest': -1, 'pay': -1, 'without': -1, 'charge': -1, 'take': -1, 'said': -1, 'would': -1, 'received': -1, 'collection': -1, 'time': -1, 'put': -1, 'one': -1, 'address': -1, 'chase': -1, 'provide': -1, 'debt': -1, 'could': -1, 'provided': -1, 'check': -1, 'home': -1, 'e

In [19]:
vocab_list = sorted(vocab.keys(), key = lambda x:x)
#print('vocab_list = ', vocab_list)
    
for i,term in enumerate(vocab_list):
    vocab[term] = i

In [20]:
print(type(vocab))
print(vocab)


<class 'dict'>
{'first': 34, 'us': 95, 'service': 83, 'like': 49, 'company': 17, 'told': 93, 'send': 81, 'back': 6, 'amount': 4, 'remove': 75, 'payment': 66, 'information': 41, 'make': 52, 'please': 68, 'dispute': 27, 'days': 25, 'due': 28, 'account': 1, 'number': 62, 'name': 58, 'work': 98, 'call': 11, 'financial': 33, 'sent': 82, 'stated': 87, 'day': 24, 'able': 0, 'mortgage': 57, 'original': 64, 'bill': 9, 'phone': 67, 'customer': 22, 'need': 59, 'another': 5, 'fee': 31, 'got': 38, 'late': 47, 'statement': 88, 'file': 32, 'removed': 76, 'loan': 50, 'never': 60, 'process': 69, 'letter': 48, 'know': 45, 'see': 80, 'business': 10, 'credit': 21, 'times': 92, 'still': 89, 'help': 39, 'modification': 53, 'get': 36, 'interest': 44, 'pay': 65, 'without': 97, 'charge': 13, 'take': 90, 'said': 79, 'would': 99, 'received': 74, 'collection': 16, 'time': 91, 'put': 72, 'one': 63, 'address': 2, 'chase': 14, 'provide': 70, 'debt': 26, 'could': 20, 'provided': 71, 'check': 15, 'home': 40, 'even': 2

In [21]:
s[0:100]

'received capital one charge card offer accepted limit card used charge card right card capital one s'

In [22]:
print(k[0:10],"\n\nLength of k %s" % len(k))

['received', 'capital', 'one', 'charge', 'card', 'offer', 'accepted', 'limit', 'card', 'used'] 

Length of k 6732517


In [23]:
d1 = d[1:100]

In [24]:
print('vocab:')
print(sorted(vocab.items())[:1000])


vocab:
[('able', 0), ('account', 1), ('address', 2), ('agency', 3), ('amount', 4), ('another', 5), ('back', 6), ('balance', 7), ('bank', 8), ('bill', 9), ('business', 10), ('call', 11), ('card', 12), ('charge', 13), ('chase', 14), ('check', 15), ('collection', 16), ('company', 17), ('complaint', 18), ('contact', 19), ('could', 20), ('credit', 21), ('customer', 22), ('date', 23), ('day', 24), ('days', 25), ('debt', 26), ('dispute', 27), ('due', 28), ('even', 29), ('every', 30), ('fee', 31), ('file', 32), ('financial', 33), ('first', 34), ('full', 35), ('get', 36), ('going', 37), ('got', 38), ('help', 39), ('home', 40), ('information', 41), ('informed', 42), ('insurance', 43), ('interest', 44), ('know', 45), ('last', 46), ('late', 47), ('letter', 48), ('like', 49), ('loan', 50), ('made', 51), ('make', 52), ('modification', 53), ('money', 54), ('month', 55), ('monthly', 56), ('mortgage', 57), ('name', 58), ('need', 59), ('never', 60), ('new', 61), ('number', 62), ('one', 63), ('original',

In [25]:
print(len(vocab))

100
